In [43]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    # model_name="gpt-4o-mini-2024-07-18",
    temperature=0.1,
    # streaming=True, 
    # callbacks=[StreamingStdOutCallbackHandler()]
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120, #비용
    return_messages=True,
)

examples = [
    {
        "movie": "해리포터",
        "answer": "🧙‍♂️✨🦉",
    },
    {
        "movie": "인셉션",
        "answer": "🌀💤💡",
    },
    {
        "movie": "타이타닉",
        "answer": "🚢💔🌊",
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{movie}를 emoji 3개로 표현해줘"),
        ("ai", "{answer}")
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 영화를 단 3개의 emoji로 표현하는 전문가입니다."),
        MessagesPlaceholder(variable_name="history"),
        example_prompt,
        ("human", "{movie_name}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm

def invoke_chain(movie_name):
    result = chain.invoke({"movie_name": movie_name})
    memory.save_context(
        {"input": movie_name}, 
        {"output": result.content})
    print(result.content
          )
    

In [44]:
invoke_chain("타이타닉")

🚢💏🌊


In [46]:
invoke_chain("해리포터")

🧙‍♂️⚡️🔮


In [51]:
memory.load_memory_variables({})['history'][0].content 
# 먼저 질문한 영화가 뭔지 

'타이타닉'